### 概要

LDA是一种非监督机器学习技术，可以用来识别大规模文档集或者语料库中的潜在隐藏的主题信息。该方法假设每个词是由背后的一个潜在隐藏的主题中抽取出来。

对于语料库中的每篇文档，LDA定义了如下的生成过程：
    
        1、对每一篇文档，从主题分布抽取一个主题
        2、从上述被抽到的主题所对应的单词分布中抽取一个单词
        3、重复1和2，直到遍历文档中的每一个单词。
      
      
LDA又可以看出是一个混合模型，每篇文章是由多个主题混合而成。
<img src="img/lda9.png">

### 前置数学知识

#### Gamma函数

简单来说Gamma函数就是实数的阶乘。 $\Gamma(x) = ∫_0^{+∞}e^{-t}*t^{x-1}dt, (x>0)$

#### 二项式分布

<img src="img/lda10.png">

#### Beta函数

<img src="img/lda12.png">

#### Beta分布

<img src="img/lda11.png">

Beta函数就是Beta分布的一个归一化因子，Beta分布又是关于分布的分布，也就是他的定义域本身也是一个概率值，（0,1）区间嘛，对吧。

#### Beta分布的期望

<img src="img/lda13.png">

#### 多项式分布
<img src="img/lda14.png">

#### 狄利克雷分布

<img src="img/lda15.png">

$\Delta$是一个归一化因子。

#### 参数估计

1、什么是参数估计？
    参数估计顾名思义就是对概率分布的参数进行估计。

2、为什么要做参数估计？
    我们知道很多分布，包括离散分布（二项式分布、多项式分布）和连续分布（正太分布）都是由几个参数决定的，所以
    我们将参数估计出来了，也就是知道了这个分布，也就是能够进行预测概率了。


3、参数估计方法？
    频率派：参数是个定值，用最有方法来估，搞出一个损失函数（似然函数），令其最大，求参数。
  贝叶斯派：参数同样是个分布（通常称为先验分布），我们根据观测值就得到了似然（P(X|θ)，所以要求参数（准确来说是求参数的概率分布）的话，我们就是求后验(P(θ|X)，啥意思，就是在观测值的这个条件下，求参数的最大后验分布。实际上，在分批训练的时候，每一批数据，都可以产生一个后验，而这个后验可以作为下一批数据的先验，这样形成online trainning，这样做能节省内存，支持大数据训练数据。

<img src="img/2019-07-29_134442.png">

### Gibbs 采样推导

#### Unigram模型

   一篇文档的生成概率，简单来说可以用二项式分布来表示：
   
   $p(W) = p(w_1)p(w_2)...p(w_n) = \prod_{t=0}^V{p_t^{n_t}}$,$n_t$表示V(i)出现的次数（词频）。
   
   其中$\sum_{t=1}^V{p_t} = 1$
   
   由于没有考虑单词之间的词序，也成为了词袋模型。

#### 第一个疑问？ LDA的目标是什么？

<img src="img/2019-07-29_151258.png">

上面的p(w)是一个k^n问题，解空间太大的，根本无法直接求出。所以用MCMC方法。

#### M-H算法

<img src="img/2019-07-29_152515.png">

注意M-H的采样平稳分布，最终的分布时p(w,z)这个联合分布哦

#### Gibbs 采样的predict

<img src = "img/2019-08-02_114830.png">

#### LDA应用

##### 1、文档相似性
    利用HL距离或者JS距离计算

##### 2、文档打标签
    找出每个文档概率最大的主题下的topn的词，即为该文档标签

##### 3、新闻个性化推荐
       LDA+LR，具体思路是，LDA训练新闻语料，得到θ矩阵，每篇文章对应的主题分布，然后用LR得出主题数K和用户点击或者不点击的关系，y = sigmoid(W*X + b), W是权重值，X表示主题向量，y是标签（用户是否点击该新闻）
       
       另一种推荐方法
<img src="img/2019-07-30_093104.png">

##### 4、topic rank 主题是否显著或者特色排序
    首先构建一个垃圾主题文档分布topic-doc，$\theta_{lj} = [1/D, 1/D, ..., 1/D]$,总共有D个分量，注意这里是topic-doc哦，然后求出每个主题下的topic-doc向量θ，将θ与$\theta_{lj}$计算距离，距离越远，表示该主题越有特色，对吧。同理

<img src="img/2019-07-30_100805.png">

##### 5、word rank

    主题下最能表达该主题的词，主要应用是协助文本分类器。 
<img src="img/2019-07-30_102755.png">

    不过可以结合topic-rank中的score进行优化

### 并行化Gibbs采样法的LDA

topic_id = [0,1,2,...,K-1]


    nw[word_id][topic_id]  词对应的主题数 V * K
    nwsum[topic_id]  每个主题下的词的个数 K * 1
    nd[doc_id][topic_id] 每篇文档属于该主题的个数 M * K
    ndsum[doc_id] 每篇文档的词的个数 M * 1


// gibbs 采样算法

    public int sampling(int m, int n){

        // remove z_i from the count variable
        int topic = trnModel.z[m].get(n);
        int w = trnModel.data.docs[m].words[n]; // word在词表中的位置

        trnModel.nw[w][topic] -= 1;
        trnModel.nd[m][topic] -= 1;
        trnModel.nwsum[topic] -= 1;
        trnModel.ndsum[m] -= 1;

        double Vbeta = trnModel.V * trnModel.beta;
        double Kalpha = trnModel.K * trnModel.alpha;

        // do multinominal sampling via cumulative method
        for (int k = 0; k < trnModel.K; k++){
            trnModel.p[k] = (trnModel.nw[w][k] + trnModel.beta)/(trnModel.nwsum[k] + Vbeta) *
                    (trnModel.nd[m][k] + trnModel.alpha)/(trnModel.ndsum[m] + Kalpha);
        }

        // cumulate multinomial parameters
        // 累加法，计算下一个z轴
        for (int k = 1; k < trnModel.K; k++){
            trnModel.p[k] += trnModel.p[k - 1];
        }

        // scaled sample because of unnormalized p[]
        double u = Math.random() * trnModel.p[trnModel.K - 1];

        for (topic = 0; topic < trnModel.K; topic++){
            if (trnModel.p[topic] > u) //sample topic w.r.t distribution p
                break;
        }

        // add newly estimated z_i to count variables
        trnModel.nw[w][topic] += 1;
        trnModel.nd[m][topic] += 1;
        trnModel.nwsum[topic] += 1;
        trnModel.ndsum[m] += 1;

        return topic;
     }

主要的思路还是如何并行化更新nw、nwsum、nd、ndsum这4个矩阵。

<img src="img/2019-07-30_110410.png">

按行拆分的意思是，将文档分为多份，按列拆分的意思是，每个对文档进行拆分，简单来说就是一个文档一行。按列拆分会将一篇文档拆分很多份。

#### AD-LDA

按列拆分思路，将一篇文档拆成不同部分分散到不同机器。支持map-reduce处理。

有误差且仍然没有解决耗内存大的问题

#### spark-lda

<img src="img/2019-07-30_133502.png">

spark-lda的基本思路是，以更新上面4个矩阵不相互依赖为前提，将整个数据集划分成P分。那什么情况下会不相互依赖呢？？？

其实很简单，在语料中同一行（也就是同一篇文档），或者同一列（同一个word_id表示的词，词不在同一篇文档中）

比如：<img src="img/2019-07-30_135730.png">

划十字线上的词不可以被并行的同时执行。

所以整个算法就是：将数据划分成P * P份，P表示并发数。

比如：<img src="img/2019-07-30_140922.png"> <img src="img/2019-07-30_140954.png">

同时选择哪些块并发执行？？？ 一般有两种方式，八皇后法和对角线法（腾讯peacoak采用此法，简单来说就是对角线上的块肯定是独立的，也就是可以并发执行，好像有点类似于求行列式。）

大概的计算思路是： 组内并行，组间串行。这就要求，小组并行执行完一个迭代后，要将计算的数据nw和nd同步到下一个小组。

nw和nd、ndsum都是均分为P份，小组内各自负责更新自己的nw_i，nd_i,ndsum_i，不过，nwsum[topic_id]肯定会有冲突，为啥？因为我们的组内并行时

肯定会同时更新某一个主题，这个概率是非常大，所以误差也就出在这个统计量上。所以当完成一次迭代后，我们要对nwsum进行全局更新。

nwsum[k] = nwsum[k] + ∑(nwsum_i[k] - nwsum[k]), 当然其他三个统计量也要合并。

总结:
    <img src="img/2019-07-30_144508.png">

### 变分贝叶斯推断

#### 指数分布族
<img src="img/2019-07-31_100447.png">

    x表示随机变量
    η表示自然参数
    u(x)表示充分统计量
    a(η)是对数配分函数，实际上就是对数归一化函数
    
Dirichlet 分布的指数分布族形式：

<img src="img/2019-07-31_100745.png">

#### 重要公式

$\frac{d(a(η))}{d(η)} = E(u(x)$

<img src="img/2019-07-31_101135.png">
<img src="img/2019-07-31_101219.png">

#### 充分统计量 
<img src="img/2019-07-31_101343.png">
<img src="img/2019-07-31_101532.png">
<img src="img/2019-07-31_101604.png">

#### 拉格朗日-欧拉方程--- 泛函分析
<img src = "img/2019-07-31_101837.png">
<img src = "img/2019-07-31_102034.png">

### 变分贝叶斯

所谓变分贝叶斯法就是用一个已知的、搞的定的分布Q(Z) 去近似带隐变量的概率条件分布P(Z|X)。

怎么衡量近似呢？ 用KL距离公式，$D_{KL}(P||Q) = \sum_i{ln(\frac{P(i)}{Q(i)}) * P(i)}$



变分法的LDA概率图模型

<img src="img/2019-07-31_103134.png">
<img src="img/2019-07-31_103241.png">
<img src="img/2019-07-31_103542.png">

#### 变分法模型

<img src="img/2019-07-31_103708.png">

先看分子：
<img src="img/2019-07-31_103826.png">
    
    第一项:
<img src="img/2019-07-31_103958.png">

    N个词，每一次都是从已知主题Z之后，再对应的W的概率，这个概率保存在β_KV矩阵中
    
综合：
<img src="img/2019-07-31_104351.png">
<img src="img/2019-07-31_104743.png">
<img src="img/2019-07-31_104806.png">

#### 概率图表示
<img src = "img/2019-07-31_110338.png">

### 有个疑问？？？
为什么α、β是语料级参数，而θ是文档集参数。

如果一篇文章看做是上帝抛骰子，那么假设第一个坛子里装的是文档-主题的骰子，本来呢，按照二项分布的做法，骰子应该只有一个，假设有K个面（主题数），我把这个骰子抛3次（假设这篇文当有3个主题）即可，对吧，但是呢？？？贝叶斯们认为，不行，你这个骰子不能是固定的，应该也是有各种各样的骰子，我们是从一个坛子里抽出这个骰子，再来抛。所以呢，θ就是每个文档生成的时候，选择一个骰子，这样它肯定是文档级别的，此时的α就是θ的先验呗，Dir（alpha），就是说我这个骰子的形状（各个面的概率大小）是由Dir（alpha）而来的。

β呢？？？ 可以这么理解，因为我们的语料词表V呢，大小是固定的，而主题数K呢也是固定的，所以说，它一开始就固定了，与你语料中具体每篇文章生成的步骤是无关的。对吧，所以说他是语料级的。

继续看分母

<img src="img/2019-07-31_111924.png">
    
    隐变量z，实际上被消去了，只有两个参数矩阵 θ和β（在Gibbs中是φ） 

<img src = "img/2019-07-31_112735.png">

    从坛子中抽一个文档-主题骰子。每个面（代表主题K）的概率就知道了。
    

<img src = "img/2019-07-31_113308.png">

    类似于每篇文档中的每个词的生成概率是一个混合模型，它是由各个主题下对改词的概率之和。

<img src = "img/2019-07-31_113441.png">


我们说LDA的变分模型非常难求，为啥？主要是7.2公式中的分母难求。

<img src="img/2019-07-31_114314.png">

主要是7.12中对z求和中，出现了θ和β，根本求不出来

### 变分推断怎么来的

<img src="img/2019-07-31_115304.png">
<img src="img/2019-07-31_115114.png">
<img src="img/2019-07-31_123840.png">

### 简化版的变分LDA

<img src="img/2019-07-31_134836.png">

<img src="img/2019-07-31_134927.png">

<img src="img/2019-07-31_135019.png">

<img src="img/2019-07-31_140209.png">

根据前面的7.15公式，我们有：
<img src="img/2019-07-31_140238.png">

#### 下界
<img src="img/2019-07-31_140352.png">

<img src="img/2019-07-31_140415.png">

<img src="img/2019-07-31_140445.png">

<img src="img/2019-07-31_140624.png">
        
        注意，为了简化，我们把q分布中省略了γ和φ
        
        L(γ,φ,α,β)就是下界，我们的目标就是max 这个下界。


##### 第一项 P(θ|α) 是Dirichlet分布
<img src="img/2019-07-31_141153.png">

    θ不是 ~Dir(α) 吗？？ 怎么变成了 ~Dir(γ)了
    
    注意，这里是对q()函数求期望，分布的参数当然也就变成γ了
    
<img src="img/2019-07-31_141229.png">

##### 第二项 P(z|θ) 
<img src="img/2019-07-31_141837.png">

<img src="img/2019-07-31_141900.png">

##### 第三项 P(w| z, β) 
<img src="img/2019-07-31_142051.png">


##### 第四项 q(θ| γ) 
<img src="img/2019-07-31_143247.png">

<img src="img/2019-07-31_143300.png">

##### 第五项 q(z|φ) 
<img src="img/2019-07-31_143613.png">

<img src="img/2019-07-31_143635.png">


##### 汇总：
<img src="img/2019-07-31_143656.png">


##### 变分EM求解
<img src="img/2019-07-31_143937.png">

变分推断EM算法希望通过“变分推断(Variational Inference)”和EM算法来得到LDA模型的文档主题分布和主题词分布。首先来看EM算法在这里的使用，我们的模型里面有隐藏变量θ,β,z
，模型的参数是α,η
。为了求出模型参数和对应的隐藏变量分布，EM算法需要在E步先求出隐藏变量θ,β,z
的基于条件概率分布的期望，接着在M步极大化这个期望，得到更新的后验模型参数α,η

。

问题是在EM算法的E步，由于θ,β,z
的耦合，我们难以求出隐藏变量θ,β,z

的条件概率分布，也难以求出对应的期望，需要“变分推断“来帮忙，这里所谓的变分推断，也就是在隐藏变量存在耦合的情况下，我们通过变分假设，即假设所有的隐藏变量都是通过各自的独立分布形成的，这样就去掉了隐藏变量之间的耦合关系。我们用各个独立分布形成的变分分布来模拟近似隐藏变量的条件分布，这样就可以顺利的使用EM算法了。

当进行若干轮的E步和M步的迭代更新之后，我们可以得到合适的近似隐藏变量分布θ,β,z
和模型后验参数α,η

，进而就得到了我们需要的LDA文档主题分布和主题词分布。

可见要完全理解LDA的变分推断EM算法，需要搞清楚它在E步变分推断的过程和推断完毕后EM算法的过程。


    https://www.cntofu.com/book/85/nlp/lda/vi-em.md

### 注意

1、VB中的$Z_d,n$是 与 Gibbs中的Z不同，VB中的Z是 一个三维矩阵 d,n,i  ，在主题上求和。而Gibbs中的Z就是表示哪个主题而已。好像也差不多，但是呢？ VB中的Z对应的 --> φ是一个word-topic概率分布,就是说第d篇文档中第n个词对应的主题i的概率，∑k(φ_d,n,i) = 1。

啥意思？gibbs中的z是由doc-topic 骰子抛出来的，参数只有d 个，因为是d个骰子，骰子上的面（概率）是φ，而vb中的φ它在每个d，n位置上都有k个概率值，所以他有d * n * k 个值。

我们的φ既然是个概率分布，那么它就应该做归一化，但是在做VB估计的时候其实φ是单独做的，每一个k，所以呢?  最后求完一个词后要做归一化

2、关于似然L(α,β，φ，γ) 其实也是变分下界。VB的思想就是在E-step时将两个变分参数 φ和γ求max，然后在M-ste时遍历完所有的doc后，再求L关于α和β的max。α,β是模型参数，是语料级参数，而φ和γ是变分模型q(φ,γ)，是文档及参数。

3、关于参数α的求解，无法直接用拉格朗日求出，因为导数无法直接求出，所以只能用（牛顿）迭代法。